#  Part 2 - More  Uncommon Data Cleaners for your Machine or Deep Learning Project

## import local

In [1]:
!conda list

# packages in environment at /Users/brucecottman/anaconda3/envs/paso:
#
# Name                    Version                   Build  Channel
_ipyw_jlab_nb_ext_conf    0.1.0                    py37_0  
absl-py                   0.7.0                    pypi_0    pypi
alabaster                 0.7.12                   py37_0  
anaconda-client           1.7.2                    py37_0  
anaconda-navigator        1.9.6                    py37_0  
anaconda-project          0.8.2                    py37_0  
ansimarkup                1.4.0                    pypi_0    pypi
appnope                   0.1.0                    py37_0  
appscript                 1.0.1            py37h1de35cc_1  
asn1crypto                0.24.0                   py37_0  
astor                     0.7.1                    pypi_0    pypi
astroid                   2.1.0                    py37_0  
astropy                   3.1              py37h1de35cc_0  
astunparse                1.6.2                    pypi_0    py

In [2]:
from __future__ import print_function
__file__ = !cd .. ;pwd
__file__ = __file__[0]
__file__

'/Users/brucecottman/Documents/PROJECTS/paso'

In [3]:
import sys
from random import random
sys.path.append(__file__)
sys.path

['/Users/brucecottman/Documents/PROJECTS/paso/lessons',
 '/Users/brucecottman/anaconda3/envs/paso/lib/python37.zip',
 '/Users/brucecottman/anaconda3/envs/paso/lib/python3.7',
 '/Users/brucecottman/anaconda3/envs/paso/lib/python3.7/lib-dynload',
 '',
 '/Users/brucecottman/.local/lib/python3.7/site-packages',
 '/Users/brucecottman/anaconda3/envs/paso/lib/python3.7/site-packages',
 '/Users/brucecottman/anaconda3/envs/paso/lib/python3.7/site-packages/aeosa',
 '/Users/brucecottman/anaconda3/envs/paso/lib/python3.7/site-packages/IPython/extensions',
 '/Users/brucecottman/.ipython',
 '/Users/brucecottman/Documents/PROJECTS/paso']

In [4]:
sys.path.append(__file__)
sys.path

['/Users/brucecottman/Documents/PROJECTS/paso/lessons',
 '/Users/brucecottman/anaconda3/envs/paso/lib/python37.zip',
 '/Users/brucecottman/anaconda3/envs/paso/lib/python3.7',
 '/Users/brucecottman/anaconda3/envs/paso/lib/python3.7/lib-dynload',
 '',
 '/Users/brucecottman/.local/lib/python3.7/site-packages',
 '/Users/brucecottman/anaconda3/envs/paso/lib/python3.7/site-packages',
 '/Users/brucecottman/anaconda3/envs/paso/lib/python3.7/site-packages/aeosa',
 '/Users/brucecottman/anaconda3/envs/paso/lib/python3.7/site-packages/IPython/extensions',
 '/Users/brucecottman/.ipython',
 '/Users/brucecottman/Documents/PROJECTS/paso',
 '/Users/brucecottman/Documents/PROJECTS/paso']

In [5]:
import sys
from random import random
from tqdm import tqdm
import numpy as np
import pandas as pd
from pandas_summary import DataFrameSummary

import warnings
warnings.filterwarnings("ignore")
import multiprocessing
OMP_NUM_THREADS=1
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import cm

%reload_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib
import seaborn as sns
color = sns.color_palette()
sns.set(rc={'figure.figsize':(6,4)})
from loguru import logger

## Executive Summary

In Part 2, **Part 2 - More  Uncommon Data Cleaners for your Machine or Deep Learning Project**,  I will cover type and format transformative data cleaners.


Almost all **paso**, including all the functions discussed here, require that **paso** data
cleaning steps (such as removal of Null and NA values)
has been done previous their use.

These Data Cleaners are detailed in the lesson [Part 1 - Uncommon Data Cleaners for your Real-World Machine Learning Projects](https://github.com/bcottman/paso/blob/master/lessons/lesson-2.ipynb).

## Background Story

In our previous projects, we found ourselves spending the majority of our  time on cleaning the data with very similar or slightly different data cleaners.

On the first project, we overspent quite a bit in  time in data cleaning development. So much so, that at the start of the 2nd project, project management had divided data cleaning into three categories: same, less than 33% different, and greater than 33% different. 

These categories were defined for two major reasons:
- As a team we were able, as data cleaner needs were identified, to eliminate a lot of "recode the wheel" and add to our data cleaners package the greater than 33% different category.
- Our development time estimations became more precise.

After 2 and a half years, we are on our 6th project and find that, so far, that less than 1 out of 10 data cleaners fall into the third, more developer-time expensive, category. More importantly, we can now spend the majority of our time on finding a good-enough approach to the machine learning (and sometimes deep learning) solution.

Your experience may be different, as my current domain is fixed-income financial data for asset-allocation, balancing, trends and risks. (Ok, yes, some other really obscure stuff that I won't mention or use here.)

Now you may ask why I am telling this story. Because.. as we went down this path, I evolved from a machine learning scientist to a software architect and software engineer. 

I now get brought in to fix a in-production services as well as accomplish initial architecture design work and code development. I enjoy going from development to production. Sometimes I get to write and share some of our accomplishments.

I sincerely hope you are able to use some of our work. All code, for functions used here, can be found at [https://github.com/bcottman/paso/toutil.py](https://github.com/bcottman/paso/toutil.py).

## Enough story already, what can you show me!

The use of different combinations of cleaning strategies (pipelines) is an important goal of **paso**.

Discussion will be divided into the following major segments:
- First we create dataset we can clean and change. 
- Following this,  each cleaner is demonstrated by acting on the reated   dataset.
- Then we show an invocation capability that mixes well with pipelines: chaining off the pandas data frame. This is familiar to R development model. We find it much easier to review code review than  sklearn pipelines. We would even argue that chaining is easier than traditional pipelining to understand.
- Finally, we summarize paso's data cleaners and what we have planned for future articles and paso releases.

The code for this lesson is given in [here](https://github.com/bcottman/paso/blob/master/lessons/) and [here](https://github.com/bcottman/paso/blob/master/paso/pre/toutil.py) files. The best way to get the entire paso package is to git clone the base.  ``pip``will not work as there are text-based files (descriptions), 

## to_util: to_DataFrame

    paso.pre.toutil.toDataFrame(X: pd.DataFrame,
        columns: List = None, 
        inplace: bool = True, 
        verbose: bool = True) 
        -> pd.DataFrame:
        
    Parameters:
        X: DataFrame
            Attempt to convert python datatype argument into pandas ataframe.

        columns:     
            The column names to  be used for new DataFrame. If number of column names given is less than number of column names needed, then they will generared as c_0...c_(n-1), where n is the number of missing column names.

        verbose: default: True) Logging is on.

        Returns: DataFrame
         
Code was based on `` willmcginnis's convert_input()``.

**paso** is a package in which all function and methods require a **pandas** DataFrame.

Since **paso** functions and methods consume only **pandas** dataframes, we need a function to transform various python data-types and while also delivering **paso's** basic services. This function is ``paso.to_util.toDataFrame``. 

This function is the exception to "**paso** first argument is a DataFrame " rule, because it transforms a 1-D or 2-D list, tuple, csr_matrix, numpy array, or 1d **pandas** Series  into a  **pandas** DataFrame.

You also have ``paso.to_util.isDataFrame(X) -> bool`` to check if the object is or is not a ``pandas.DataFrame``.

Now you can use the method ``pandas.DataFrame``, if you don't require **paso's** basic services, such as logging and caching. I would recommend that you not do this, but you have the un-pythonic choice to do so.

A good practice is to make your dataset of type ``DataFrame`` at the start of your pipeline. Changes by the **paso** function will be kept in the DataFrame's change of state (``inplace=True``). This will happen
though-out the pipeline of your experimental run to maximize
speed of completion and minimize memory usage. 

If you need to refer to the original DataFrame, then you can do something similar to this:

    <dataset_name>_original = <dataset_name>.copy()
    
However, you should be warned that keeping the original is going to take memory.  Almost always you input the original from a local file or non-local file addressed by URL.  

It is up-to you to keep around a copy of the original dataset. Upon looking over your approach and goal(s), you will probably find you do not need to a copy of the original dataset.

You can see the size of any object in bytes with:

In [6]:
from sys import getsizeof
getsizeof(pd.DataFrame())

24

This shows that an empty DataFrame has a size of 24 bytes.

Now, create a artificial dataset, ``df_of_ints``, a numpy array of `int`s.

In [36]:
from paso.base import raise_PasoError  
from paso.pre.toutil import toDataFrame
arr = np.ndarray(shape=(1000000,6), dtype=int, order='F')
df_of_ints = toDataFrame(arr)

2019-12-12 13:29:08.164 | INFO     | paso.pre.toutil:toDataFrame:112 - toDataFrame  with 
column names: Index(['c_0', 'c_1', 'c_2', 'c_3', 'c_4', 'c_5'], dtype='object')


` toDataFrame(arr, columns=['a','b'])`

 can specify first two column names,for the new DataFrame.

In [37]:
df_of_ints = toDataFrame(arr, columns=['a','b'])
df_of_ints.shape

2019-12-12 13:29:22.981 | INFO     | paso.pre.toutil:toDataFrame:112 - toDataFrame  with 
column names: Index(['a', 'b', 'c_2', 'c_3', 'c_4', 'c_5'], dtype='object')


(1000000, 6)

Notice how the first two column names were given and ``toDataFrame``detected this and created the next 4 that were needed.

This time we invoke ``toDataFrame`` while keeping **paso** logging off. Logging is only turned off for the scope of the function invoked.

In [38]:
df_of_ints = toDataFrame(arr, columns=['X','yyyyyy'],verbose =False)
df_of_ints.columns

Index(['X', 'yyyyyy', 'c_2', 'c_3', 'c_4', 'c_5'], dtype='object')

## to_util: toCategory

     paso.pre.toutil.toCategory(
        X: pd.DataFrame,
        bool_: bool = True,
        int_: bool = True,
        object_: str = True,
        inplace: bool = True,
        verbose: bool = True,
    ) -> pd.DataFrame:
    
    Parameters:
        X: dataFrame

        bool_: default :True
            If True will convert to category type.

        int_: default: True
            If True will convert to category type.

        object_: default: True
            If True will convert to category type.

        inplace: default: True
            True: change DataFrame arg in-place"

            False:  return new resulting DataFrame
            
        verbose: default: True Logging is on.
    
    Returns: resulting DataFrame

This function transforms any boolean, object or integer values of 
of a pandas DataFrame to category type values.

The exception is continuous (``float`` or ``datetime``) values which are returned un-transformed. 

If you want to convert continuous or datetime types to
category then use ``ContinuoustoCategory`` or ``DatetimetoComponents``
before **paso** (step) ``toCategory``.

Note:
Assumes **paso**  data
cleaning steps (such as removal of Null and NA values) have
been done previous to this step.

``datetime`` features should call ``toDatetimeComponents()``
previous to this step so that ``datetime`` components (which are of type
``np.number``) can be converted to ``category``.

The default
behavior of this step is NOT to convert ``datetime`` to ``category``.

In [39]:
from paso.base import raise_PasoError  
from paso.pre.toutil import toCategory
df_cat = toCategory(df_of_ints,inplace=False)

2019-12-12 13:36:58.748 | INFO     | paso.pre.toutil:toCategory:197 - toCategory integer feature converted : X
2019-12-12 13:36:58.768 | INFO     | paso.pre.toutil:toCategory:197 - toCategory integer feature converted : yyyyyy
2019-12-12 13:36:58.786 | INFO     | paso.pre.toutil:toCategory:197 - toCategory integer feature converted : c_2
2019-12-12 13:36:58.802 | INFO     | paso.pre.toutil:toCategory:197 - toCategory integer feature converted : c_3
2019-12-12 13:36:58.814 | INFO     | paso.pre.toutil:toCategory:197 - toCategory integer feature converted : c_4
2019-12-12 13:36:58.824 | INFO     | paso.pre.toutil:toCategory:197 - toCategory integer feature converted : c_5


Looks like ``df_of_ints`` is unchanged, which is what want in this case, `inplace=False`
.

In [11]:
df_of_ints.dtypes

X         int64
yyyyyy    int64
c_2       int64
c_3       int64
c_4       int64
c_5       int64
dtype: object

Now it looks like all values of ``df_of_ints`` are of type int converted to tpye category, `inplace=True`.

In [12]:
df_of_ints.toCategory(inplace=True)
df_of_ints.dtypes

2019-12-12 09:39:37.193 | INFO     | paso.pre.toutil:toCategory:197 - toCategory integer feature converted : X
2019-12-12 09:39:37.214 | INFO     | paso.pre.toutil:toCategory:197 - toCategory integer feature converted : yyyyyy
2019-12-12 09:39:37.233 | INFO     | paso.pre.toutil:toCategory:197 - toCategory integer feature converted : c_2
2019-12-12 09:39:37.251 | INFO     | paso.pre.toutil:toCategory:197 - toCategory integer feature converted : c_3
2019-12-12 09:39:37.269 | INFO     | paso.pre.toutil:toCategory:197 - toCategory integer feature converted : c_4
2019-12-12 09:39:37.282 | INFO     | paso.pre.toutil:toCategory:197 - toCategory integer feature converted : c_5


X         category
yyyyyy    category
c_2       category
c_3       category
c_4       category
c_5       category
dtype: object

## Chaining

I am guessing some of the readers (not you) are a little bothered by ``df_of_ints.toCategory(inplace=True)``. If so, you will be more bothered by this:

In [13]:
toDataFrame(arr, columns=['a','b']).toCategory(verbose=False).dtypes

2019-12-12 09:39:37.359 | INFO     | paso.pre.toutil:toDataFrame:112 - toDataFrame  with column names: Index(['a', 'b', 'c_2', 'c_3', 'c_4', 'c_5'], dtype='object')


a      category
b      category
c_2    category
c_3    category
c_4    category
c_5    category
dtype: object

If you are not bothered or even puzzled about this fuss, it is because you know that a DataFrame is a ``python`` class and you call methods off a class. You can even *chain* methods after a class or class instance.

In this case, I *chained*  the two methods ``.toCategory().dtypes`` off the DataFame output by `toDataFrame`
.

All the metods of class ``Cleaners`` and functions of ``toutils.py``of **paso** require the 1st argument to be a
 DataFrame instance. This enables them to be methods of a DataFrame instance by using a decorator found in the package **pandas-flavor**.
 
Chaining can make it very easy to understand, because operations are invoked left to right. That is the same way you read (English and many other languages).

Some people prefer to chain instead of using **sklearn's Pipelines**.  You will notice how our chaining is slightly different in syntax than **sklearn's Pipeline** chaining.

Our chaining is very close to the look-and-feel of ``R``.   We will use both invocation methods from now on.

## toutil: toDatetimeComponents

     paso.pre.toutil.toDatetimeComponents(
        X: pd.DataFrame,
        drop: bool = True,
        components: list = [],
        prefix: bool = True,
        inplace: bool = True,
        verbose: bool = True) -> pd.DataFrame:
        
    Parameters:
        
        X: DataFrame

        drop: default: True
            If True then the datetime feature/column will be removed.

        components:: default: []
            list of column(feature) names for which datetime components
            are created.

        prefix: default: True
            If True then the feature will be the prefix of the created datetime component fetures. The posfix will be _<component> to create the new feature column <feature>_<component>.

            if False only first _PREFIX_LENGTH_ characters of feature string will be used to
            create the new feature name/column <featurename[0:_PREFIX_LENGTH_]>_<component>.

        components:list default:
            ['Year', 'Month', 'Week', 'Day','Dayofweek'
            , 'Dayofyear','Elapsed','Is_month_end'
            , 'Is_month_start', 'Is_quarter_end'
            , 'Is_quarter_start', 'Is_year_end', 'Is_year_start']

            or set components to one or component names in a list
            Must be components names from default list.
            
        inplace: default: True
            True: change DataFrame arg in-place"

            False: return new resulting dataframe
            
        verbose: default: True Logging is on.
    

    Returns:DataFrame
            toDatetimeComponents transformed into datetime feature components

   Raises:
   
        ValueError: if any dt_features = [].

        ValueError: if any feature has NA values.
        
if X[[dt_features]] is not of datetime type  (such as ``object`` type)
then there **IS NO** attempt to coerce X[[dt_features]] to ``datetime`` type.

It is best if raw data field
is read/input in as ``datetime`` rather than ``object``. Another way, is to convert
dataframe column using

    X[feature] = pd.datetime(X[feature])

For many reasons, all resulting in better predictive power, you can break a ``datetime`` into ``int`` components.  You can go further and convert ``datetime`` ``int`` components into categorical-type components.

Why would you want to this? Well it leads to an advanced encoding tactic, that is discussed in detail in the paper [Entity Embeddings of Categorical Variables ]( https://`arxiv.org/pdf/1604.06737.pdf) 
and [Entity Embeddings of Categorical Variables in Neural Networks](https://towardsdatascience.com/decoded-entity-embeddings-of-categorical-variables-in-neural-networks-1d2468311635a).

The latter article states:

    "Categorical variables are known to hide and mask lots of interesting information in a data set and many times they might even be the most important variables in a model. A good data scientist should be capable of handling such variables effectively and efficiently. If you are a smart data scientist, you’d hunt down the categorical variables in the data set, and dig out as much information as you can."

You probably see were this going. If I have some non-categorical features, such as column of ``datetimes``, I will need a function to turn datetime into categorical(s) feature(s). 

I  will cover how to use categorical features as embedding vectors in an upcoming article.

You can see what datetime components are available with ``datetimeComponents``.

In [14]:
from paso.pre.toutil import datetimeComponents
datetimeComponents()

['Year',
 'Month',
 'Week',
 'Day',
 'Dayofweek',
 'Dayofyear',
 'Elapsed',
 'Is_month_end',
 'Is_month_start',
 'Is_quarter_end',
 'Is_quarter_start',
 'Is_year_end',
 'Is_year_start']

I first create dataset to can show what ``toDatetimeComponents  `` does.

In [15]:
from datetime import datetime, timedelta

rc = 94598; cc = 1; acc =13
darr = np.arange(datetime(1751,7,1),
                 datetime(2010,7,1),
                 timedelta(days=1)).reshape(rc,cc)

#%debug
from paso.pre.toutil import toDatetimeComponents
#toDataFrame(darr,labels = [])
toDataFrame(darr,columns = []).toDatetimeComponents(inplace=False).head(2)


2019-12-12 09:39:37.601 | INFO     | paso.pre.toutil:toDataFrame:112 - toDataFrame  with column names: Index(['c_0'], dtype='object')
2019-12-12 09:39:37.609 | INFO     | paso.pre.toutil:toDatetimeComponents:346 - datetime feature component added: c_0_Year
2019-12-12 09:39:37.614 | INFO     | paso.pre.toutil:toDatetimeComponents:346 - datetime feature component added: c_0_Month
2019-12-12 09:39:37.621 | INFO     | paso.pre.toutil:toDatetimeComponents:346 - datetime feature component added: c_0_Week
2019-12-12 09:39:37.627 | INFO     | paso.pre.toutil:toDatetimeComponents:346 - datetime feature component added: c_0_Day
2019-12-12 09:39:37.633 | INFO     | paso.pre.toutil:toDatetimeComponents:346 - datetime feature component added: c_0_Dayofweek
2019-12-12 09:39:37.640 | INFO     | paso.pre.toutil:toDatetimeComponents:346 - datetime feature component added: c_0_Dayofyear
2019-12-12 09:39:37.794 | INFO     | paso.pre.toutil:toDatetimeComponents:346 - datetime feature component added: c_0_

,c_0_Year,c_0_Month,c_0_Week,c_0_Day,c_0_Dayofweek,c_0_Dayofyear,c_0_Elapsed,c_0_Is_month_end,c_0_Is_month_start,c_0_Is_quarter_end,c_0_Is_quarter_start,c_0_Is_year_end,c_0_Is_year_start
0,1751,7,26,1,3,182,1694609792,False,True,False,True,False,False
1,1751,7,26,2,4,183,1694696192,False,False,False,False,False,False


In this case, ``datetimeComponents`` transformed each datetime value into 13 ``int`` components. It then transformed those ``int`` types into ``category`` types.

## toutil: toContinuousCategory

    def toContinuousCategory(
        X: pd.DataFrame,
        features:List=[],
        drop: bool = True,
        inplace: bool = True,
        verbose: bool = True,
        integer: bool = True,
        floaty: bool = True,
        quantile: bool = True,
        nbin: int = 10,
    ) -> pd.DataFrame:
    
    Parameters:
        X:pd.DataFrame

        drop: default:True
            do not keep original features.

        integer: default:True
            set integer=False if not continuous and not to 
            transform into category.

        floaty: default:True
            set floaty=False if not continuous and not to 
            transform into category.

        quantile: default: True
            use quantile bin.
            quantile is similar to v/(maxy-miny), works on any scale.
            False, use fixed-width bin. miny,maxy arguments are ignored.

        nbin:int (default: 10)

    Returns: DataFrame

    Raises:
        TypeError('"ContinuoustoCategory:inplace: requires boolean type.")

``toContinuousCategory`` transforms any continuous float, or integer values of
a pandas dataframe to category values by grouping into bins.

Binning, also known as quantization is used for
transforming continuous numeric features
(``np.number`` type) into ``category`` type.

Each bin represents a range of continuous numeric values.

Specific strategies of binning data include fixed-width
(``quantile=False``) and adaptive binning (``quantile = True``).

No matter the type of the input dataset it  will be returned
as ``DataFrame``. If you want to set the feature names, call ``toDataFrame`` before this function.

Fixed-width bin, only works, WITHOUT SCALING, with datasets with multiple features for tree-based models such as CART, random forest, xgboost, lightgbm, catboost,etc. Namely linear, log, SVM, neural nets, etc. won't work correctly.

**Statistical problems with linear binning.**

Binning increases type I and type II error; (simple proof is that as number
of bins approaches infinity then information loss approaches zero).
In addition, changing the number of bins will alter the bin distrution shape,
unless the distribution is uniformLY FLAT.

**Quantile binning can only be used with a singular data set.**

Transforming a Continuous featuree ino a Category feature based on percentiles (QUANTILES) is WRONG
if you have a train and test data sets. Quantiles are based on the data set and will be different unless
each data set distribution shape is the same. In the limit there are only two bins,
then almost no relationship can be modeled. We are essentially doing a t-test.

**if there are nonlinear or even nonmonotonic relationships between features**

If you need linear binning, not quantile, use
``quantile=False`` .

**If you want Quantile-binning.**

Despite the above warnings, your use case may require. quantile-binning.
Quantile based binning is a faily good strategy to use for adaptive binning.
Quantiles are specific values or cut-points which partition
the continuous valued distribution of a feature into
discrete contiguous bins or intervals. Thus, q-Quantiles
partition a numeric attribute into q equal (percetage-width) partitions.

Well-known examples of quantiles include the 2-Quantile ,median,
divides the data distribution into two equal (percetage-width) bins, 4-Quantiles,
,standard quartiles, 4 equal bins (percentage-width) and 10-Quantiles,
deciles, 10 equal width (percentage-width) bins.

You should note that the new features resulting from quantile-binning are appended with ``q``. While new features resulting from fixed-width-binning are appended with ``w``.


Tranform continuous float features into category and drop the original columns.

In [41]:
from paso.pre.toutil import toContinuousCategory
nc = 6
nr = 1000000
delta = 0.1
farr = np.arange(0,(nr*delta*nc),delta, dtype=float).reshape(nr,nc)
toDataFrame(farr).toContinuousCategory(drop=True).head(n=2)


2019-12-12 15:27:27.638 | INFO     | paso.pre.toutil:toDataFrame:112 - toDataFrame  with 
column names: Index(['c_0', 'c_1', 'c_2', 'c_3', 'c_4', 'c_5'], dtype='object')
2019-12-12 15:27:28.204 | INFO     | paso.pre.toutil:toContinuousCategory:533 - toContinuousCategory features:: Index(['c_0', 'c_1', 'c_2', 'c_3', 'c_4', 'c_5'], dtype='object')


,c_0q,c_1q,c_2q,c_3q,c_4q,c_5q
0,"(-0.001, 59999.94]","(0.099, 60000.04]","(0.199, 60000.14]","(0.299, 60000.24]","(0.399, 60000.34]","(0.499, 60000.44]"
1,"(-0.001, 59999.94]","(0.099, 60000.04]","(0.199, 60000.14]","(0.299, 60000.24]","(0.399, 60000.34]","(0.499, 60000.44]"


Tranform continuous float features into category and leave the original columns.

In [17]:
toDataFrame(farr).toContinuousCategory(quantile=False,nbin=3,drop=False)\
.head(n=2)

2019-12-12 09:39:38.578 | INFO     | paso.pre.toutil:toDataFrame:112 - toDataFrame  with column names: Index(['c_0', 'c_1', 'c_2', 'c_3', 'c_4', 'c_5'], dtype='object')
2019-12-12 09:39:38.896 | INFO     | paso.pre.toutil:toContinuousCategory:533 - toContinuousCategory features:: Index(['c_0', 'c_1', 'c_2', 'c_3', 'c_4', 'c_5'], dtype='object')


,c_0,c_1,c_2,c_3,c_4,c_5,c_0fw,c_1fw,c_2fw,c_3fw,c_4fw,c_5fw
0,0.0,0.1,0.2,0.3,0.4,0.5,"(-599.999, 199999.8]","(-599.899, 199999.9]","(-599.799, 200000.0]","(-599.699, 200000.1]","(-599.599, 200000.2]","(-599.499, 200000.3]"
1,0.6,0.7,0.8,0.9,1.0,1.1,"(-599.999, 199999.8]","(-599.899, 199999.9]","(-599.799, 200000.0]","(-599.699, 200000.1]","(-599.599, 200000.2]","(-599.499, 200000.3]"


## toutil: toColumnNamesFixedLen


    def toColumnNamesFixedLen(
        oX: pd.DataFrame,
        column_length: int = 1,
        col_separator: str = "_",
        inplace: bool = True,
        verbose: bool = True,
    ) -> pd.DataFrame:
    
        Parameters:
        X: dataset

    Keywords:

        column_length:  Default: 3
            Character length for which to truncate all columns.
            The column separator value and number for duplicate
            column name does not contribute. Therefore, if all
            columns are truncated to 10 characters, the first
            distinct column will be 10 characters and the
            remaining will be 12 characters (assuming a column
            separator of one character).

        col_separator: Default: "_"
            The separator to append plus incremental Int 
            to create unique column names. Care should be 
            taken in choosing non-default col_separator 
            so as to create legal pandas column name.

        verbose: Default: True
            True: output
            False: silent

        inplace: Default: True
            True: replace 1st argument with resulting dataframe
            False:  (boolean)change unplace the dataframe X

    Returns: A pandas DataFrame with truncated column lengths.

Truncate column name to a specific length.  If
column length is shorter, then column length left 
as is.

``toColumnNamesFixedLen`` will truncate all columns to a 
given length and append a given separator 
character with the index of duplicate columns, except
for the first distinct column name.

In [18]:
toDataFrame([[1,2,3,4]],columns=[
     "long_name",
     "another_long_name",
     "another_longer_name",
     "data",
]).toColumnNamesFixedLen(column_length=5)

2019-12-12 09:39:38.981 | INFO     | paso.pre.toutil:toDataFrame:112 - toDataFrame  with column names: Index(['long_name', 'another_long_name', 'another_longer_name', 'data'], dtype='object')
2019-12-12 09:39:38.982 | INFO     | paso.pre.toutil:toColumnNamesFixedLen:669 - toColumnNamesFixedLen features:: ['long_', 'anoth', 'anoth_1', 'data']


,long_,anoth,anoth_1,data
0,1,2,3,4


In [19]:
toDataFrame([[1,2,3,4]],columns=[
     "long_name",
     "another_long_name",
     "another_longer_name",
     "data",
]).toColumnNamesFixedLen(column_length=2,column_separator= '=')

2019-12-12 09:39:39.051 | INFO     | paso.pre.toutil:toDataFrame:112 - toDataFrame  with column names: Index(['long_name', 'another_long_name', 'another_longer_name', 'data'], dtype='object')
2019-12-12 09:39:39.052 | INFO     | paso.pre.toutil:toColumnNamesFixedLen:669 - toColumnNamesFixedLen features:: ['lo', 'an', 'an=1', 'da']


,lo,an,an=1,da
0,1,2,3,4


## Functional Programming - Creating readable pipelines with chaining

In this section. I will show a functional programming style in python. This style is also called *chaining* and should be very familiar to those who use **R**.

I am going to create one dirty dataset so we can show some of our cleaners in a chain (pipeline).

In [30]:
arr = np.ndarray(shape=(1000000,6), dtype=int, order='F')
dataset = toDataFrame(arr)
dataset['date'] = pd.to_datetime('11/30/1956')

2019-12-12 09:46:58.614 | INFO     | paso.pre.toutil:toDataFrame:112 - toDataFrame  with column names: Index(['c_0', 'c_1', 'c_2', 'c_3', 'c_4', 'c_5'], dtype='object')


In [33]:
dataset.toDatetimeComponents(inplace=False).toColumnNamesFixedLen(column_length=12).toContinuousCategory(quantile=False,nbin=10,drop=False)\
.head(n=3)

2019-12-12 10:18:49.817 | INFO     | paso.pre.toutil:toDatetimeComponents:346 - datetime feature component added: date_Year
2019-12-12 10:18:49.861 | INFO     | paso.pre.toutil:toDatetimeComponents:346 - datetime feature component added: date_Month
2019-12-12 10:18:49.920 | INFO     | paso.pre.toutil:toDatetimeComponents:346 - datetime feature component added: date_Week
2019-12-12 10:18:49.962 | INFO     | paso.pre.toutil:toDatetimeComponents:346 - datetime feature component added: date_Day
2019-12-12 10:18:50.013 | INFO     | paso.pre.toutil:toDatetimeComponents:346 - datetime feature component added: date_Dayofweek
2019-12-12 10:18:50.060 | INFO     | paso.pre.toutil:toDatetimeComponents:346 - datetime feature component added: date_Dayofyear
2019-12-12 10:18:50.092 | INFO     | paso.pre.toutil:toDatetimeComponents:346 - datetime feature component added: date_Elapsed
2019-12-12 10:18:50.139 | INFO     | paso.pre.toutil:toDatetimeComponents:346 - datetime feature component added: date_

,c_0,c_1,c_2,c_3,c_4,c_5,date_Year,date_Month,date_Week,date_Day,...,c_2fw,c_3fw,c_4fw,c_5fw,date_Yearfw,date_Monthfw,date_Weekfw,date_Dayfw,date_Dayofwefw,date_Dayofyefw
0,0,0,0,0,0,0,1956,11,48,30,...,"(-0.0002, 0.0]","(-0.0002, 0.0]","(-0.0002, 0.0]","(-0.0002, 0.0]","(1955.609, 1956.0]","(10.998, 11.0]","(47.99, 48.0]","(29.994, 30.0]","(3.9992, 4.0]","(334.933, 335.0]"
1,0,0,0,0,0,0,1956,11,48,30,...,"(-0.0002, 0.0]","(-0.0002, 0.0]","(-0.0002, 0.0]","(-0.0002, 0.0]","(1955.609, 1956.0]","(10.998, 11.0]","(47.99, 48.0]","(29.994, 30.0]","(3.9992, 4.0]","(334.933, 335.0]"
2,0,0,0,0,0,0,1956,11,48,30,...,"(-0.0002, 0.0]","(-0.0002, 0.0]","(-0.0002, 0.0]","(-0.0002, 0.0]","(1955.609, 1956.0]","(10.998, 11.0]","(47.99, 48.0]","(29.994, 30.0]","(3.9992, 4.0]","(334.933, 335.0]"


Notice *chaining* is not transitive. The final resulting dataframe will depend on the ordering of chain of functions invocations.

For example, the functions in the chain are the same, but the  order of the functions are different. The final result is a different dataframe. In this case, the column names are different.

In [34]:
dataset.toDatetimeComponents(inplace=False)\
.toContinuousCategory(quantile=False,nbin=10,drop=False)\
.toColumnNamesFixedLen(column_length=12)\
.head(n=3)

2019-12-12 10:29:00.193 | INFO     | paso.pre.toutil:toDatetimeComponents:346 - datetime feature component added: date_Year
2019-12-12 10:29:00.237 | INFO     | paso.pre.toutil:toDatetimeComponents:346 - datetime feature component added: date_Month
2019-12-12 10:29:00.297 | INFO     | paso.pre.toutil:toDatetimeComponents:346 - datetime feature component added: date_Week
2019-12-12 10:29:00.340 | INFO     | paso.pre.toutil:toDatetimeComponents:346 - datetime feature component added: date_Day
2019-12-12 10:29:00.391 | INFO     | paso.pre.toutil:toDatetimeComponents:346 - datetime feature component added: date_Dayofweek
2019-12-12 10:29:00.437 | INFO     | paso.pre.toutil:toDatetimeComponents:346 - datetime feature component added: date_Dayofyear
2019-12-12 10:29:00.467 | INFO     | paso.pre.toutil:toDatetimeComponents:346 - datetime feature component added: date_Elapsed
2019-12-12 10:29:00.513 | INFO     | paso.pre.toutil:toDatetimeComponents:346 - datetime feature component added: date_

,c_0,c_1,c_2,c_3,c_4,c_5,date_Year,date_Month,date_Week,date_Day,...,c_2fw,c_3fw,c_4fw,c_5fw,date_Yearfw,date_Monthfw,date_Weekfw,date_Dayfw,date_Dayofwe_1,date_Dayofye_1
0,0,0,0,0,0,0,1956,11,48,30,...,"(-0.0002, 0.0]","(-0.0002, 0.0]","(-0.0002, 0.0]","(-0.0002, 0.0]","(1955.609, 1956.0]","(10.998, 11.0]","(47.99, 48.0]","(29.994, 30.0]","(3.9992, 4.0]","(334.933, 335.0]"
1,0,0,0,0,0,0,1956,11,48,30,...,"(-0.0002, 0.0]","(-0.0002, 0.0]","(-0.0002, 0.0]","(-0.0002, 0.0]","(1955.609, 1956.0]","(10.998, 11.0]","(47.99, 48.0]","(29.994, 30.0]","(3.9992, 4.0]","(334.933, 335.0]"
2,0,0,0,0,0,0,1956,11,48,30,...,"(-0.0002, 0.0]","(-0.0002, 0.0]","(-0.0002, 0.0]","(-0.0002, 0.0]","(1955.609, 1956.0]","(10.998, 11.0]","(47.99, 48.0]","(29.994, 30.0]","(3.9992, 4.0]","(334.933, 335.0]"


## Summary

Starting with Python 3.6 (PEP 484), type hints were introduced. Type hints (note: not strong type checking) make it possible to also do static type checking of Python code with downstream tools.

Using type hints, the ``toutil`` function call signatures are summarized:

    # 1
    paso.pre.toutil.toDataFrame(
            X: pd.DataFrame,
            columns: List = None, 
            inplace: bool = True, 
            verbose: bool = True) -> pd.DataFrame:
    #2
    paso.pre.toutil.toCategory(
        X: pd.DataFrame,
        bool_: bool = True,
        int_: bool = True,
        object_: str = True,
        inplace: bool = True,
        verbose: bool = True) -> pd.DataFrame:

    #3
    paso.pre.toutil.datetimeComponents():

    #4
     paso.pre.toutil.toDatetimeComponents(
        oX: pd.DataFrame,
        drop: bool = True,
        components: list = [],
        prefix: bool = True,
        inplace: bool = True,
        verbose: bool = True) -> pd.DataFrame:

    #5
    paso.pre.toutil.def toColumnNamesFixedLen(
        X: pd.DataFrame,
        column_length: int = 1,
        col_separator: str = "_",
        inplace: bool = True,
        verbose: bool = True) -> pd.DataFrame:

The code for this lesson is given in [here](https://github.com/bcottman/paso/blob/master/lessons/) and [here](https://github.com/bcottman/paso/blob/master/paso/pre/toutil.py) files.

Other lessons on **paso** are:

[**paso**'s Offering of Logging and Parameter Services for your Python Project](https://github.com/bcottman/paso/blob/master/lessons/lesson_1.ipynb)

[Part 1: Balancing and Augmenting Structured Data](https://medium.com/@dr.bruce.cottman/part-1-balancing-and-augmenting-structured-data-4ade0df38662)

[Part 1 - Uncommon Data Cleaners for your Real-World Machine Learning Projects](https://github.com/bcottman/paso/blob/master/lessons/lesson-2.ipynb).


In the future, we will cover **paso** in more depth with the following lessons:

- Chaining an entire Machine Leaning Pipeline.
- Part 2: Yet more (and better) methods for Balancing and Augmenting Structured Data.
- Part 1: Different encoding methods for categorical features.
- Part 2: Deep learning encoding for categorical features.

If you have a service or feature or see a bug, then leave the **paso** project a [note](https://github.com/bcottman/paso/issues).